# Kmeans로 뉴스데이터 clustering 하기

## 1. 형태소 분리

In [ ]:
import pandas as pd     # csv 파일 편집
import MeCab            # 전처리
import re               # 정규 표현식 (전처리)

전처리된 CSV 파일을 만든다

이미 만들었다면 PASS

In [ ]:
csv_test = pd.read_csv('./SampleData/etoday.csv')
cnt = 0
m = MeCab.Tagger()

for line in csv_test["Contents"]:
    remove_email = re.compile(r'\(([^)]+\)) | [^가-힣]+@yna\.co\.kr')     # 괄호 및 안의 내용 제거 | 이메일 주소 제거
    remove_special_char = re.compile(r'[^가-힣^A-z^0-9^ ]') # 한글, 영어, 기본 문자를 제외한 문자들 제거
    text = remove_email.sub(' ', line)
    text = remove_special_char.sub(' ', text)
    
    tagged = m.parse(text)
    s = tagged.split('\n')

    result = []
    
    for words in s:
        # MeCab 으로 형태소 분리되면 끝이 EOS 임.
        if words == 'EOS':
            break
        word, tag = words.split(',')[0].split('\t')
        d_tag = tag.split('+')
        if (d_tag[-1] != ""):
            tag = d_tag[0]

        if tag in ["NNB", "NNBC", "VV", "VA", "VX", "VCP", "VCN", "VSV", "MAG", "MAJ", "JKS", "JKC", "JKG", 
                    "JKO", "JKB", "JKV", "JKQ", "JC", "JX", "EP", "EF", "EC", "ETN", "ETM", "XPN", "XSN", 
                    "XSV", "XSA", "SF", "SE", "SSO", "SSC", "SC", "SY", "SH", "SL", "SN", "UNA", "NA"]:
            pass
        else:
            result.append(word)
    sss = ' '.join(result)
    # 데이터 프레임으로 만들고,
    df = pd.DataFrame({
        'CNo' : csv_test['CNo'][cnt],
        'Subject' : csv_test['Subject'][cnt],
        'Contents' : [sss]
    })
    # 바로 저장
    df.to_csv("./pretreatment_data/etoday_pre.csv", encoding='utf-8',mode = 'a', index=False, header=False)
    cnt += 1
    # 얼마나 했는지 확인용
    if cnt % 1000 == 0:
        print(cnt, " 완료")

## 2. 단어 임베딩

문서간에 비교를 위해, 단어를 임베딩하여 벡터화 시킬거임

두가지 방법으로 진행해보았다.

1. CountVectorizer
2. TfidfVectorizer

예상으로는 TfidfVectorizer 가 더 성능이 좋게 나올 것 같다.

In [ ]:
et_df = pd.read_csv("./pretreatment_data/etoday_pre.csv", names = ["CNo", "Subject", "Contents"])

### 1) CountVectorizer

: 단어들의 카운트(출현빈도(frequency))로 여러 문서들을 벡터화

카운트 행렬, 단어 문서 행렬

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

# deep copy
count_df = et_df[:]

n_cluster = 100

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(count_df["Contents"])

X = normalize(X)

kmeans = KMeans(n_clusters=n_cluster).fit(X)

labels = kmeans.labels_
centers = kmeans.cluster_centers_

count_df["labels"] = labels

In [ ]:
count_df.loc[count_df['labels'] == 55, ['Subject', 'labels']]

In [ ]:
count_df.loc[count_df['labels'] == 66].index

### 2) TfidfVectorizer()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

#deep copy
tfidf_df = et_df[:]
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(tfidf_df['Contents'])

tfidf_matrix = normalize(tfidf_matrix)

kmeans_tfidf = KMeans(n_clusters=100).fit(tfidf_matrix)

labels = kmeans_tfidf.labels_
centers = kmeans_tfidf.cluster_centers_

tfidf_df["labels"] = labels

In [ ]:
print(tfidf_matrix)

In [ ]:
print(tfidf_df.loc[tfidf_df['labels'] == 55, ['Subject', 'labels']])

In [ ]:
print(kmeans_tfidf.score(tfidf_matrix))

In [ ]:
# tfidf_matrix.shape[0] -> 전체 문서 갯수

im_word = []
voca = sorted(tfidf.vocabulary_)
for i in range(tfidf_matrix.shape[0]):
    # i 번째 문서에서 가중치가 가장 높은 단어를 추출
    value = tfidf_matrix[i].argmax()
    im_word.append(voca[value])
print(set(im_word))

In [ ]:
label55 = tfidf_df.loc[tfidf_df['labels'] == 55].index
word55 = []
voca = sorted(tfidf.vocabulary_)
for i in label55:
    # i 번째 문서에서 가중치가 가장 높은 단어를 추출
    value = tfidf_matrix[i].argmax()
    word55.append(voca[value])
print(word55)
print(set(word55))

In [ ]:
len(set(im_word))

In [ ]:
print(type(tfidf_matrix[0]))

In [ ]:
sorted(tfidf.vocabulary_)[10488]

In [ ]:
print(tfidf_matrix[1])

In [ ]:
tfidf_matrix[1].argmax()

In [ ]:
tfidf_matrix[0].argmin()

## 적절한 k 값 찾기

In [ ]:
print("왜곡 : {}".format(kmeans_tfidf.inertia_))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.cluster import KMeans

# we are usingh
df=et_df[:]

print(df.head())

In [ ]:
distortions = []
elbow_tfidf = TfidfVectorizer()

elbow_tfidf_matrix = elbow_tfidf.fit_transform(df['Contents'])

elbow_tfidf_matrix = normalize(elbow_tfidf_matrix)
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(elbow_tfidf_matrix)
    distortions.append(kmeanModel.inertia_)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
K = range(10,50)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(elbow_tfidf_matrix)
    distortions.append(kmeanModel.inertia_)
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, distortions[9:50], 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
K = range(1,50)
plt.figure(figsize=(16,8))
plt.plot(K, distortions[:50], 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

# 유사도 비교

바자도 KNN 이 뽑아낸 값과 비교하기 위해 사용.

In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer

NN_df = pd.read_csv("./pretreatment_data/etoday_pre.csv", names=["CNo", "Subjects", "Contents"])

tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(NN_df['Contents'])

In [204]:
def clac_sim(num):
    # target_matrix에는 문서 하나(이투데이)의 TF-IDF 값이 저장된다.
    target_matrix = tfidf.transform([NN_df._get_value(num, 'Contents')]).astype(np.float16)
    # cosin_matrix에는 유사도 결과 값들이 저장된다.
    cosine_matrix = cosine_similarity(target_matrix, tfidf_matrix)

# 보기 쉽게 맵핑하는 부분
    # news title과 id를 맵핑할 dictionary를 생성
    news2id = {}
    for i, c in enumerate(NN_df['CNo']):
        news2id[i] = c

    # id와 news title를 매핑할 dictionary를 생성
    id2news = {}
    for i, c in news2id.items():
        id2news[c] = i
# 맵핑 끝
    sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[0])]
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    # sim_scores 에는 문서 번호(CNo) 와 유사도 값이 리스트 형태로 저장됨
    # 5개 저장
    sim_scores = [(news2id[i], round(score, 4)) for i, score in sim_scores if score >= 0.7]
# 
    return sim_scores

In [206]:
clac_sim(8080)

[('E_1955225_20201027_120103.txt', 1.0),
 ('E_1953959_20201023_120202.txt', 0.844),
 ('E_1953430_20201022_120103.txt', 0.8351),
 ('E_1954665_20201026_120103.txt', 0.819),
 ('E_1951296_20201016_120102.txt', 0.8162),
 ('E_1955802_20201028_120101.txt', 0.8122),
 ('E_1949265_20201012_120102.txt', 0.8049),
 ('E_1948462_20201008_120102.txt', 0.8044),
 ('E_1947920_20201007_120103.txt', 0.7982),
 ('E_1956707_20201030_090403.txt', 0.795),
 ('E_1952934_20201021_120102.txt', 0.7932),
 ('E_1950325_20201014_120203.txt', 0.7882),
 ('E_1950655_20201015_090403.txt', 0.7848),
 ('E_1950850_20201015_120102.txt', 0.7829),
 ('E_1952466_20201020_120103.txt', 0.7818),
 ('E_1955078_20201027_090404.txt', 0.7789),
 ('E_1954060_20201023_153402.txt', 0.7755),
 ('E_1954495_20201026_090402.txt', 0.7724),
 ('E_1949786_20201013_120102.txt', 0.7692),
 ('E_1946886_20201005_120102.txt', 0.7683),
 ('E_1952785_20201021_090401.txt', 0.7676),
 ('E_1951163_20201016_090502.txt', 0.767),
 ('E_1951903_20201019_120103.txt', 0.76

# 비지도 KNN

In [3]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

# 사용 데이터 불러오기
NN_df = pd.read_csv("./pretreatment_data/etoday_pre.csv", names=["CNo", "Subjects", "Contents"])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
# Tfidf로 데이터 임베딩
nn_tf = TfidfVectorizer()

nn_tf_matrix = nn_tf.fit_transform(NN_df['Contents'])

nn_tf_matrix = normalize(nn_tf_matrix)

In [215]:
# 이웃 검색을 위한 비지도 학습기
# n_neighbors = 이웃수
# radius = 반경

neigh = NearestNeighbors(n_neighbors=10, radius=0.78)

neigh.fit(nn_tf_matrix)

NearestNeighbors(n_neighbors=10, radius=0.78)

In [191]:
print(nn_tf_matrix[0])

  (0, 26938)	0.020609648026551138
  (0, 36657)	0.02726429180559092
  (0, 3886)	0.02101533056723216
  (0, 31657)	0.04506928394524605
  (0, 38381)	0.04142633763737896
  (0, 4188)	0.045243715926478974
  (0, 24538)	0.03276154750122785
  (0, 21667)	0.03441002516659542
  (0, 18085)	0.04046811741980474
  (0, 4561)	0.04198007674542899
  (0, 32186)	0.02324455705596447
  (0, 21364)	0.021921897685280217
  (0, 32586)	0.024008100922409896
  (0, 35867)	0.03175303349724676
  (0, 37918)	0.03318222480407057
  (0, 4309)	0.028151965184440512
  (0, 26070)	0.026789263857342076
  (0, 35604)	0.02999658075961418
  (0, 9132)	0.04198007674542899
  (0, 22510)	0.03895615809418049
  (0, 7249)	0.017973194703385258
  (0, 14015)	0.01937517804311522
  (0, 6610)	0.015201894346838855
  (0, 3168)	0.013627695038187289
  (0, 14195)	0.028871811228738848
  :	:
  (0, 6969)	0.03752696678735667
  (0, 9913)	0.02798641888188761
  (0, 14014)	0.029068307406953903
  (0, 15354)	0.04506928394524605
  (0, 19227)	0.03329287437671011
  (

In [216]:
from sklearn.metrics.pairwise import cosine_similarity
# 점의 지정된 반경 내에서 이웃을 찾습니다.
nbrs = neigh.radius_neighbors(nn_tf_matrix[8080], sort_results = True)
print(len(nbrs[0][0]))

me = nbrs[1][0][0]

for n, i in enumerate(nbrs[1][0]):
    cosine_matrix = cosine_similarity(nn_tf_matrix[me], nn_tf_matrix[i])
    print('거리 : {: .4f} / 제목 : {} '.format(nbrs[0][0][n], NN_df["Subjects"][i]))
    print("유사도 : ", round(cosine_matrix[0][0], 4))

61
거리 :  0.0000 / 제목 : [시황_정오] 코스피 2344.84p, 상승세 (▲0.93p, +0.04%) 반전 
유사도 :  1.0
거리 :  0.5586 / 제목 : [시황_정오] 코스피 2365.84p, 상승세 (▲10.79p, +0.46%) 지속 
유사도 :  0.844
거리 :  0.5743 / 제목 : [시황_정오] 코스피 2353.07p, 하락세 (▼17.79p, -0.75%) 지속 
유사도 :  0.8351
거리 :  0.6016 / 제목 : [시황_정오] 코스피 2356.9p, 하락세 (▼3.91p, -0.17%) 반전 
유사도 :  0.819
거리 :  0.6063 / 제목 : [시황_정오] 코스피 2351.94p, 하락세 (▼9.27p, -0.39%) 반전 
유사도 :  0.8162
거리 :  0.6129 / 제목 : [시황_정오] 코스피 2328.77p, 하락세 (▼2.07p, -0.09%) 지속 
유사도 :  0.8122
거리 :  0.6247 / 제목 : [시황_정오] 코스피 2404.53p, 상승세 (▲12.57p, +0.53%) 지속 
유사도 :  0.8049
거리 :  0.6254 / 제목 : [시황_정오] 코스피 2394.66p, 상승세 (▲7.72p, +0.32%) 지속 
유사도 :  0.8044
거리 :  0.6353 / 제목 : [시황_정오] 코스피 2371.21p, 상승세 (▲5.31p, +0.22%) 반전 
유사도 :  0.7982
거리 :  0.6404 / 제목 : [시황_개장] 코스피 2312.86p, 외국인 순매도에 하락세 (▼13.81p, -0.59%) 
유사도 :  0.795
거리 :  0.6432 / 제목 : [시황_정오] 코스피 2364.04p, 상승세 (▲5.63p, +0.24%) 지속 
유사도 :  0.7932
거리 :  0.6509 / 제목 : [시황_정오] 코스피 2384.78p, 하락세 (▼18.37p, -0.76%) 지속 
유사도 :  0.7882
거리 :  0.6560 / 제목 : [

In [195]:
nbrs[0][0]

array([0.        , 0.44764929, 0.73577991])

In [209]:
# kneighbors : 점의 k-이웃을 찾는다.
# 
nei_list = neigh.kneighbors(nn_tf_matrix[8080])
print(nei_list[1])

[[8080 6933 6442 7573 4519 8609 2688 1986 1510 9407]]


In [210]:
from sklearn.metrics.pairwise import cosine_similarity
# me 에 기준점을 저장
me = nei_list[1][0][0]

for n, i in enumerate(nei_list[1][0]):
    # 기준점과 이웃되는 점의 코사인 유사도를 계산
    cosine_matrix = cosine_similarity(nn_tf_matrix[me], nn_tf_matrix[i])
    print('거리 : {: .4f} / 제목 : {} '.format(nei_list[0][0][n], NN_df["Subjects"][i]))
    print("코사인 유사도 : ", round(cosine_matrix[0][0], 4))

거리 :  0.0000 / 제목 : [시황_정오] 코스피 2344.84p, 상승세 (▲0.93p, +0.04%) 반전 
코사인 유사도 :  1.0
거리 :  0.5586 / 제목 : [시황_정오] 코스피 2365.84p, 상승세 (▲10.79p, +0.46%) 지속 
코사인 유사도 :  0.844
거리 :  0.5743 / 제목 : [시황_정오] 코스피 2353.07p, 하락세 (▼17.79p, -0.75%) 지속 
코사인 유사도 :  0.8351
거리 :  0.6016 / 제목 : [시황_정오] 코스피 2356.9p, 하락세 (▼3.91p, -0.17%) 반전 
코사인 유사도 :  0.819
거리 :  0.6063 / 제목 : [시황_정오] 코스피 2351.94p, 하락세 (▼9.27p, -0.39%) 반전 
코사인 유사도 :  0.8162
거리 :  0.6129 / 제목 : [시황_정오] 코스피 2328.77p, 하락세 (▼2.07p, -0.09%) 지속 
코사인 유사도 :  0.8122
거리 :  0.6247 / 제목 : [시황_정오] 코스피 2404.53p, 상승세 (▲12.57p, +0.53%) 지속 
코사인 유사도 :  0.8049
거리 :  0.6254 / 제목 : [시황_정오] 코스피 2394.66p, 상승세 (▲7.72p, +0.32%) 지속 
코사인 유사도 :  0.8044
거리 :  0.6353 / 제목 : [시황_정오] 코스피 2371.21p, 상승세 (▲5.31p, +0.22%) 반전 
코사인 유사도 :  0.7982
거리 :  0.6404 / 제목 : [시황_개장] 코스피 2312.86p, 외국인 순매도에 하락세 (▼13.81p, -0.59%) 
코사인 유사도 :  0.795


In [43]:
nei_list[1][0]

array([   0, 5928, 4836, 1724, 3577], dtype=int64)

In [71]:
nei_list[0][0][2]

1.0220781561234535

시각화

In [89]:
w_list = []
w_list.append([])
for i in  nn_tf.vocabulary_.keys():
    w_list.append(nn_tf.vocabulary_[i])

In [90]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_result = pca.fit_transform(w_list)



ValueError: setting an array element with a sequence.

In [ ]:
import time
from sklearn.manifold import TSNE

n_sne = 9000

time_start = time.time()
tsen = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsen_results = tsen.fit_transform(et_df.loc[9000, tfidf.vocabulary_.keys()].values)

print('t-SNE done! Time elapsed: {} second'.format(time.time() - time_start))